In [157]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

#第一個問題，本次跑的模型中，線性回歸，SVR和DBSCAN無法跑出結果，而以最初使用的邏輯回歸為基準，決策樹分類和回歸的表現並沒有比較好，但是隨機森林
#的表現好於邏輯回歸，而SVC，KMeans和AP的表現比邏輯回歸來的差，其中AP後三次並沒有跑出結果

#第2個問題本次以LogisticRegression為基準進行GridSearchCV找尋參數並設定後比較結果，得知其得分基本上不差於原本的結果(即有幾次比原本好，有幾次
#跟原本的結果差不多)

#本次最後是用pytorch來建立模型，跑出來的結果最後大概落於0.83~0.84之間，總體而言比某些模型來的好些

#資料前處理與特徵提取
pd.set_option('display.max_rows',None)
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Embarked'] = df['Embarked'].fillna('S')

title = pd.DataFrame()
title['Title'] = df['Name'].map(lambda name: name.split(',')[1].split('.')[0].strip())

Title_Dictionary={
                  "Capt": "Officer",
                  "Col": "Officer",
                  "Major": "Officer",
                  "Jonkheer": "Royalty",
                  "Don": "Royalty",
                  "Sir": "Royalty",
                  "Dr": "Officer",
                  "Rev": "Officer",
                  "the Countess": "Royalty",
                  "Dona": "Royalty",
                  "Mme": "Mrs",
                  "Mile": "Miss",
                  "Ms": "Mrs",
                  "Mr": "Mr",
                  "Mrs": "Mrs",
                  "Miss": "Miss",
                  "Master": "Master",
                  "Lady": "Royalty"
}
title['Title'] = title.Title.map(Title_Dictionary)
title =pd.get_dummies(title.Title)

def cleanTicket(ticket):
    ticket =ticket.replace('.','')
    ticket = ticket.replace('/','')
    ticket = ticket.split()
    ticket = map(lambda t: t.strip(),ticket)
    ticket = list(filter(lambda t: not t.isdigit(), ticket))
    if len(ticket)>0:
        return ticket[0]
    else:
        return 'xxx'
ticket = pd.DataFrame()
ticket['Ticket'] = df['Ticket'].map(cleanTicket)
ticket = pd.get_dummies(ticket['Ticket'],prefix='Ticket')

df['ismotherchild'] = np.where((df['Age']<18) | (df['Parch'] > 0),1,0)
df['isfatherchild'] = np.where((df['Age']<18) & (df['Parch'] > 0),1,0)

family = pd.DataFrame()
family['FamilySize']=df['Parch']+df['SibSp']+1
family['Family_single']= family['FamilySize'].map(lambda s:1 if s ==1 else 0)
family['Family_small']=family['FamilySize'].map(lambda s:1 if 2 <=s <= 4 else 0)
family['Family_large']=family['FamilySize'].map(lambda s:1 if 5<= s else 0)

df = pd.get_dummies(df,columns = ['Sex','Embarked'])
df1 = pd.concat([df,title,ticket,family],axis=1)
df_train = df1.drop(['Cabin','Ticket','Name'],axis=1)





In [143]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]

#用邏輯回歸做分類分析
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)

[0.83240223 0.8258427  0.79775281 0.80898876 0.87078652]


In [98]:
from sklearn.preprocessing import LabelEncoder
#用LabelEncoder方法做前處理後用邏輯回歸做分類分析
le = LabelEncoder()
dfe2['Embarkedl']= le.fit_transform(dfe2['Embarked'])
dfe2 = dfe2.drop(['Embarked'],axis = 1)
dfe2['Sexl']= le.fit_transform(dfe2['Sex'])
dfe2 = dfe2.drop(['Sex'],axis = 1)
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)

[0.78212291 0.78089888 0.78089888 0.76404494 0.81460674]


In [147]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]

#用線性回歸做分類分析(當然沒結果)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score


lin = LinearRegression()
scores = cross_val_score(lin, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)

[nan nan nan nan nan]


C:\Users\st030\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\st030\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\st030\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 106, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "C:\Users\st030\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 267, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\st030\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "C:\Users\st030\anaconda3\lib\site-packages\sklearn\metrics\_classification

In [150]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]


from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

#用決策樹分類做分類分析
dtc = DecisionTreeClassifier(max_depth = 3000, random_state = 0)
scores = cross_val_score(dtc, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)







[0.57541899 0.81460674 0.76404494 0.79213483 0.73033708]


In [151]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]

#用決策樹回歸做分類分析
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score


dtr = DecisionTreeRegressor(max_depth = 3000, random_state = 0)
scores = cross_val_score(dtr, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)


[0.76536313 0.76404494 0.75842697 0.79775281 0.7752809 ]


In [152]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]

#用隨機森林做分類分析
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


rfc = RandomForestClassifier(max_depth = 3000, random_state = 0)
scores = cross_val_score(rfc, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)

[0.78212291 0.79775281 0.82022472 0.82022472 0.83146067]


In [153]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]

#用支持向量機分類做分類分析
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score


svc = SVC(random_state=0, max_iter=3000)
scores = cross_val_score(svc, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)

[0.61452514 0.64044944 0.64606742 0.64044944 0.65168539]


In [155]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]

#用支持向量機回歸做分類分析(當然沒結果)
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score


svr = SVR( max_iter=3000)
scores = cross_val_score(svr, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)

[nan nan nan nan nan]


C:\Users\st030\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\st030\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\st030\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 106, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "C:\Users\st030\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 267, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\st030\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "C:\Users\st030\anaconda3\lib\site-packages\sklearn\metrics\_classification

In [169]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]

#用KMeans做分類分析
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score


km = KMeans( n_clusters= 1, max_iter=3000,random_state = 1)
scores = cross_val_score(km, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)

C:\Users\st030\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
C:\Users\st030\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
C:\Users\st030\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
C:\Users\st030\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Window

[0.67039106 0.55617978 0.61235955 0.59550562 0.64606742]


In [177]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]


#用AP做分類分析
from sklearn.cluster import AffinityPropagation
from sklearn.model_selection import cross_val_score


AP = AffinityPropagation( max_iter=3000,random_state = 1)
scores = cross_val_score(AP, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)

[0.66480447 0.00561798 0.         0.         0.        ]


In [180]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]

#用DBSCAN做分類分析
from sklearn.cluster import DBSCAN
from sklearn.model_selection import cross_val_score


DB = DBSCAN()
scores = cross_val_score(DB, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)

[nan nan nan nan nan]


C:\Users\st030\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\st030\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\st030\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 106, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "C:\Users\st030\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 261, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\st030\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 71, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'DBSCAN' object has no attribute 'predict'

  warnings.warn(
C:\Users\st030\anaconda3\lib\site-packages

In [186]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]
#在邏輯回歸做分類分析的條件下用GridSearchCV找出最好的模型參數以跑出最理想之結果並對設定前與設定後做比較
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

X= train_X
y = train_y.values.ravel()

log = LogisticRegression(random_state=0, max_iter=3000)
param_grid={'penalty':['l1','l2' ],
           'C': [0.001,0.01,0.1,1,10,100],
           'solver': ['liblinear', 'saga'],
          }
gs = GridSearchCV(log, param_grid,cv = 5, scoring = 'accuracy')
gs.fit(X,y)
print(gs.best_params_)
print(gs.best_estimator_)

log1 = LogisticRegression(penalty = 'l2',C= 1.0,solver = 'liblinear',random_state=0, max_iter=3000)

scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)
scores1 = cross_val_score(log1, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores1)


C:\Users\st030\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\st030\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\st030\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\st030\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\st030\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\st030\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter 

{'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
LogisticRegression(C=100, max_iter=3000, random_state=0, solver='liblinear')
[0.83240223 0.8258427  0.79775281 0.80898876 0.87078652]
[0.83240223 0.83146067 0.80337079 0.80898876 0.87078652]


In [196]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]
#利用pytorch建立模型逕行分類分析並得出模型的精準度
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
class TitanicDataset(Dataset):
    def __init__(self,df):
        self.X=df.drop('Survived',axis =1).values
        self.y = df['Survived'].values
    
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class TitanicModel(nn.Module):
    def __init__(self, input_dim):
        super(TitanicModel, self).__init__()
        self.fc1=nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,2)
        self.relu = nn.ReLU()
        self.dropout= nn.Dropout(0.2)
        
    def forward(self,x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

def train(model,train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    for i ,(inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs,targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        predicted = torch.argmax(outputs,1)
        train_acc += torch.sum(predicted == targets.data)
        
    train_loss = train_loss/ len(train_loader.dataset)
    train_acc = train_acc/len(train_loader.dataset)
    return train_loss, train_acc
    
def validate(model, val_loader,  criterion, device):
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(val_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs.float())
            loss = criterion(outputs,targets)
            val_loss += loss.item() * inputs.size(0)
            predicted = torch.argmax(outputs,1)
            val_acc += torch.sum(predicted == targets.data)
    val_loss = val_loss/ len(val_loader.dataset)
    val_acc = val_acc/ len(val_loader.dataset)
    return val_loss, val_acc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TitanicModel(input_dim = 54).to(device)
optimizer = optim.Adam(model.parameters(),lr =0.001)
criterion = nn.CrossEntropyLoss()
train_dataset = TitanicDataset(df_train)
train_loader = DataLoader(train_dataset, batch_size = 32,shuffle = True)
val_dataset = TitanicDataset(df_train)
val_loader =  DataLoader(val_dataset, batch_size = 32,shuffle = False)
for epoch in range(100):
    train_loss, train_acc=train(model,train_loader,optimizer,criterion, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    print(f"Epoch {epoch +1}: Train Loss={train_loss:.4f} Train Acc={train_acc:.4f} Val Loss={val_loss:.4f} Val Acc={val_acc:.4f}")

        



Epoch 1: Train Loss=1.7636 Train Acc=0.5522 Val Loss=0.6275 Val Acc=0.6633
Epoch 2: Train Loss=0.8864 Train Acc=0.6049 Val Loss=0.6253 Val Acc=0.6599
Epoch 3: Train Loss=0.7076 Train Acc=0.6285 Val Loss=0.6390 Val Acc=0.6263
Epoch 4: Train Loss=0.7079 Train Acc=0.6319 Val Loss=0.6297 Val Acc=0.6364
Epoch 5: Train Loss=0.6792 Train Acc=0.6296 Val Loss=0.6484 Val Acc=0.6678
Epoch 6: Train Loss=0.6672 Train Acc=0.6263 Val Loss=0.6274 Val Acc=0.6734
Epoch 7: Train Loss=0.6430 Train Acc=0.6510 Val Loss=0.6103 Val Acc=0.6902
Epoch 8: Train Loss=0.6440 Train Acc=0.6420 Val Loss=0.6123 Val Acc=0.7026
Epoch 9: Train Loss=0.6268 Train Acc=0.6700 Val Loss=0.5931 Val Acc=0.6869
Epoch 10: Train Loss=0.6144 Train Acc=0.6723 Val Loss=0.5916 Val Acc=0.6880
Epoch 11: Train Loss=0.6191 Train Acc=0.6678 Val Loss=0.5900 Val Acc=0.6891
Epoch 12: Train Loss=0.6255 Train Acc=0.6655 Val Loss=0.5871 Val Acc=0.6891
Epoch 13: Train Loss=0.6107 Train Acc=0.6846 Val Loss=0.5885 Val Acc=0.6992
Epoch 14: Train Loss=